In [1]:
import nltk
import os
from nltk.corpus import stopwords
import string
import csv
from scipy.spatial.distance import cosine, cdist
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from nltk.probability import FreqDist


path='D:\\College\\Semester 9\\NLP\\Lab\\FinalProject\\Tweets'
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
DocsTerms={}
stop_words=set(stopwords.words('english')) 
i=1
for root, dirs, files in os.walk(path):
    for file in files: # looping only over first 20,000 files
        
        with open(os.path.join(root,file), 'r', encoding="utf8") as f:
            reader = csv.reader(f)
            for row in reader:
                if not row[0] in DocsTerms :
                   # CheckifRetweet(row[10])
                    if " RT " not in row[10] or " rt " not in row[10] and  len(row[10]) > 20 : 
                        key =row[0] 
                        DocsTerms[key]=[]
                        DocsTerms[key].append(row[1])
                        LowerVocab=row[10].lower()
                        noLinkSent=re.sub(r'http\S+','',LowerVocab,flags=re.MULTILINE)
                        words = tokenizer.tokenize(noLinkSent)
                        stemmedVocab=[]
                        for word in words:
                            stemmedVocab.append(ps.stem(word))
                        filtered_sent=[w for w in stemmedVocab if not w in stop_words]
                        DocsTerms[key].append(filtered_sent)

DocsTerms.pop('tweet_id')


train={}
test={}
i=0
for key, value in DocsTerms.items():
    if i<=11712 :
        train[key]=[]
        train[key].append(value[0])
        train[key].append(value[1])
    else:
        test[key]=[]
        test[key].append(value[0])
        test[key].append(value[1])
    i=i+1


TfidfVectorizerFunc = TfidfVectorizer()
CountVectorizerFunc = CountVectorizer()
sentences = []
twitterId=[]
semitment=[]
for key, value in train.items():
    s=""
    twitterId.append(key)
    semitment.append(value[0])
    for word in  value[1]:
        s+=word+" "
    sentences.append(s)
    

tfidf = TfidfVectorizerFunc.fit_transform(sentences)

sentences_test = []
twitterId_test=[]
semitment_test=[]

for key, value in test.items():
    s=""
    twitterId_test.append(key)
    semitment_test.append(value[0])
    for word in  value[1]:
        s+=word+" "
    sentences_test.append(s)


           

In [2]:
from sklearn.naive_bayes import MultinomialNB
cats = ['neutral','positive','negative']
clf = MultinomialNB(alpha=1)
clf.fit(tfidf, semitment)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [3]:
import numpy as np
tfidf_test = TfidfVectorizerFunc.transform(sentences_test)
predictions = clf.predict(tfidf_test)


In [4]:
from sklearn import metrics
print(metrics.f1_score(semitment_test, predictions, average='micro'))

0.727994227994228


In [8]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(tfidf, semitment) 
predictions_neigh = neigh.predict(tfidf_test)
print(metrics.f1_score(semitment_test, predictions_neigh, average='micro'))

0.7297979797979797


In [7]:
from sklearn.ensemble import RandomForestClassifier
randomClassifier = RandomForestClassifier(random_state=0)
randomClassifier.fit(tfidf, semitment) 
predictions_randomClassifier = randomClassifier.predict(tfidf_test)
print(metrics.f1_score(semitment_test, predictions_randomClassifier, average='micro'))



0.7784992784992785


c:\users\yasmi\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [3]:
import re
def CheckifRetweet(tweetText):
    regexp = re.compile(r'\b[R|r]t\b')
    if regexp.search(tweetText):
        print("yes")
    
    